In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import skimage as ski
import scipy.sparse
import time

In [ ]:
def imnoise(g, noise=1, sigma=0.1, theta=0.01, p_l1=0.25, p_in=0.9):
    
    # noise: 1 ... Gaussian, 2 ... S&P, 3 ... Gamma
    # sigma: standard deviation of Gaussian noise (mean=0)
    # theta: variance of Gamma noise (mean=1)
    # p_l1: fraction of outliers in S&P noise
    # p_in: fraction of missing pixels for inpainting
    
    # make a copy of g
    f = np.copy(g)
    M,N = g.shape

    if noise == 1:
        # Gaussian noise, mean = 0
        f = f + np.random.randn(M,N)*sigma
    elif noise == 2:
        # Salt and Pepper noise, a fraction of p_l1 is set either to 0 or 1
        n = np.random.rand(M,N)
        f[n <= p_l1/2] = 0.0
        f[n >= (1-p_l1/2)] = 1.0
    elif noise == 3:
        # multipliative Gamma noise, mean = 1
        k = 1/theta # ensure that mean=1
        f = f*np.random.gamma(k,theta,(M,N))
    elif noise == 4:
        # A fraction of p_in of missing pixels
        n = np.random.rand(M,N)
        f[n <= p_in] = np.nan
    return f

In [ ]:
def forward_differences(M,N):
    row = np.arange(0,M*N)
    dat = np.ones(M*N)
    col = np.arange(0,M*N).reshape(M,N)
    col_xp = np.hstack([col[:,1:], col[:,-1:]])
    col_yp = np.vstack([col[1:,:], col[-1:,:]])
    
    FD1 = scipy.sparse.coo_matrix((dat, (row, col_xp.flatten())), shape=(M*N, M*N))- \
          scipy.sparse.coo_matrix((dat, (row, col.flatten())), shape=(M*N, M*N))

    FD2 = scipy.sparse.coo_matrix((dat, (row, col_yp.flatten())), shape=(M*N, M*N))- \
          scipy.sparse.coo_matrix((dat, (row, col.flatten())), shape=(M*N, M*N))
    
    FD = scipy.sparse.vstack([FD1, FD2])
    
    return FD

In [ ]:
def l22(u,f):
    return 0.5*np.sum((u-f)**2)

def l1(u,f):
    return np.sum(np.abs(u-f))

def entropy(u,f):
    return np.sum(u-f*np.log(u))

def inpaint(u,f):
    return 0.0

def prox_l22(u, f, tau):
    return (u + tau*f)/(1+tau)

def prox_l1(u, f, tau):
    return f + np.maximum(0.0, np.abs(u-f)-tau)*np.sign(u-f)

def prox_entropy(u, f, tau):
    return (u-tau+np.sqrt((tau-u)**2 + 4*tau*f))/2.0

def prox_inpaint(u, f, idx):
    u[idx] = f[idx]
    return u

def proj_inf_l2(p, tau):
    # size must be (K,N), l2 over K, inf over N
    norm_p = np.sqrt(np.sum(p**2, axis=0, keepdims=True))
    p /= np.maximum(1, norm_p/tau)
    return p

def proj_inf(p, tau):
    return np.clip(p, -tau, tau)

In [ ]:
def tv_pd(f, dataterm=1, lamb=1.0, maxit=1000, check=100, verbose=0):
    # implementation with sparse matrix
    # dataterm: 1 ... l_2^2, 2 ... l_1, 3 ... entropy, 4 ... inpainting
    
    M,N = f.shape
    f = f.reshape(M*N)
    u = np.zeros(M*N)

    # dual variable
    p = np.zeros(M*N*2)

    # make nabla operator
    D = forward_differences(M,N)

    # primal and dual step size
    # tau * sigma * L^2 = 1
    L = np.sqrt(8)
    tau = 1/L    
    sigma = 1/tau/L**2
    theta = 1.0

    if dataterm == 4:
        idx = ~np.isnan(f)
    
    t0 = time.time()
    for it in range(0,maxit):

        # remeber old
        ui = u.copy()

        # primal update
        u -= tau*(D.T@p)
    
        # proximal maps
        if dataterm == 1:
            u = prox_l22(u,f,tau)
        
        if dataterm == 2:
            u = prox_l1(u,f,tau)

        if dataterm == 3:
            u = prox_entropy(u,f,tau)

        if dataterm == 4:
            u = prox_inpaint(u,f,idx)
            
        # overrelaxation
        ui = u + theta*(u-ui)

        if dataterm == 1:
            theta = 1/np.sqrt(1+tau)
            sigma = sigma/theta;
            tau = tau*theta;
        
        # dual update
        p += sigma*(D@ui)

        # projection
        p = p.reshape(2,M*N)
        p = proj_inf_l2(p, lamb)
        p = p.reshape(2*M*N)
        
        if verbose > 0:
            if it%check == check-1:
                TV = np.sum(np.sqrt(np.sum(((D@u).reshape(2,M*N))**2, axis=0)))
                
                if dataterm == 1:
                    DT = l22(u,f)
                elif dataterm == 2:
                    DT = l1(u,f)
                elif dataterm == 3:
                    DT = entropy(u,f)
                elif dataterm == 4:
                    DT = inpaint(u,f)
                    
                E = lamb*TV + DT

                print("iter = ", it,
                      ", tau = ", "{:.3f}".format(tau),
                      ", sigma = ", "{:.3f}".format(sigma),
                      ", time = ", "{:.3f}".format(time.time()-t0),
                      ", TV = ", "{:.3f}".format(TV),
                      end="\r")
                
    return u.reshape(M,N)

In [ ]:
# Load image
#g = ski.io.imread("watercastle.png")/255.0

# Add noise to the image
# noise: 1 ... Gaussian, 2 ... S&P, 3 ... Gamma, 4 ... Inpainting
#noise=1
#f = imnoise(g, noise=noise)

g = ski.io.imread("disks.png")/255.0
f = g.copy()
noise=1

In [ ]:
# Solve the TV model
# dataterm: 1 ... l_2^2, 2 ... l_1, 3 ... entropy, 4 ... inpainting

lamb_tv = 4.0
u_tv = tv_pd(f, dataterm=noise, lamb=lamb_tv, verbose=1)

plt.figure(1, figsize=(10,5))

plt.subplot(121)
plt.imshow(f, cmap="gray")

plt.subplot(122)
plt.imshow(u_tv, cmap="gray")